In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageChops, ImageStat, ImageDraw
import pandas as pd
import glob
import face_recognition
import fitz
from pathlib import Path
import shutil
import openpyxl
from itertools import chain

import os
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
from tensorflow.keras import applications
import tensorflow as tf
from tensorflow.keras.models import save_model
import tensorflow.keras.layers as L
from sklearn.model_selection import train_test_split
import cv2
import tqdm
from skimage import io

## Предобработка и загрузка файлов

In [1]:
path ="/Users/Desktop/Python/Passports" (здесь Ваш путь до pdf-сканов документов) 
gPDF=glob.glob('path/*.pdf')

SyntaxError: invalid syntax (939149366.py, line 1)

### Загрузка пути до всех pdf-файлов

In [ ]:
gPDF=glob.glob('pdf/*.pdf')
len(gPDF)

### Функция преобразования каждой страницы pdf-файла в изображение

In [ ]:
def extract_images_from_pdf(pdf):
    count = 0
    for tpdf in pdf:
        name = Path(tpdf).stem
        doc=fitz.open(tpdf)
        for i in range(len(doc)):
            for img in doc.get_page_images(i):
                xref=img[0]
                pix = fitz.Pixmap(doc,xref)
                if pix.n < 5:
                    pix.save(f'image_from_pdf/{name}p%s-%s.png' % (i,xref))
                else:
                    pix1 = fitz.Pixmap(fitz.csRGB, pix)
                    pix1.save(f'image_from_pdf/{name}p%s-%s.png' % (i,xref))
                    pix1 = None
                pix = None
                count+=1
    return f'Found {count} images'

In [ ]:
# Применение функции
extract_images_from_pdf(gPDF)

### Получаем путь до всех изображений полученных из pdf-файлов

In [ ]:
g=glob.glob('image_from_pdf/*.png')
#g

### Функция для распознавания лица на изображении

In [ ]:
def face_recog_pdf(gimage):
    count = 0
    for timage in gimage:
        name = Path(timage).stem
        img = face_recognition.load_image_file(timage)
        test_loc = face_recognition.face_locations(img)
        for f in test_loc:
            top, right,bottom, left = f
            face_img = img[top:bottom,left:right]
            pil_img = Image.fromarray(face_img)
            pil_img.save(f'pdf_img/{name}_face_{count}.png')
            count+=1
    return f'Found {count} face(s) in this photos'

In [ ]:
# Применение функции
face_recog_pdf(g)

In [ ]:
# Архивирование изображений с лицами - если нужно
shutil.make_archive('/home/datalab/пока_не_кейсы', 'zip', '/home/datalab/pdf_img')

In [ ]:
# Получаем путь до всех изображений с распознанными лицами
photo = glob.glob('pdf_img/*.png')
#photo

## Автоэнкодеры

### Применение функции для преобразования изображений в вектора для метода автоэнкодирования

In [ ]:
def image2array(filelist – путь до папки с фотографиями):
    image_array = []
    for image in filelist[:200]:
        img = io.imread(image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224,224))
        image_array.append(img)
    image_array = np.array(image_array)
    image_array = image_array.reshape(image_array.shape[0], 224, 224, 3)
    image_array = image_array.astype('float32')
    image_array /= 255
    return np.array(image_array)

train_data = image2array(filelist)
print("Length of training dataset:", train_data.shape)

### Применение функции для построения автоэнкодера

In [ ]:
IMG_SHAPE = x.shape[1:]
def build_deep_autoencoder(img_shape, code_size):
    H,W,C = img_shape
    # encoder
    encoder = tf.keras.models.Sequential() # инициализация модели
    encoder.add(L.InputLayer(img_shape)) # добавление входного слоя, размер равен размеру изображения
    encoder.add(L.Conv2D(filters=32, kernel_size=(3, 3), activation='elu', padding='same'))
    encoder.add(L.MaxPooling2D(pool_size=(2, 2)))
    encoder.add(L.Conv2D(filters=64, kernel_size=(3, 3), activation='elu', padding='same'))
    encoder.add(L.MaxPooling2D(pool_size=(2, 2)))
    encoder.add(L.Conv2D(filters=128, kernel_size=(3, 3), activation='elu', padding='same'))
    encoder.add(L.MaxPooling2D(pool_size=(2, 2)))
    encoder.add(L.Conv2D(filters=256, kernel_size=(3, 3), activation='elu', padding='same'))
    encoder.add(L.MaxPooling2D(pool_size=(2, 2)))
    encoder.add(L.Flatten())
    encoder.add(L.Dense(code_size))

    # decoder
    decoder = tf.keras.models.Sequential()
    decoder.add(L.InputLayer((code_size,)))
    decoder.add(L.Dense(14*14*256))
    decoder.add(L.Reshape((14, 14, 256)))
    decoder.add(L.Conv2DTranspose(filters=128, kernel_size=(3, 3), strides=2, activation='elu', padding='same'))
    decoder.add(L.Conv2DTranspose(filters=64, kernel_size=(3, 3), strides=2, activation='elu', padding='same'))
    decoder.add(L.Conv2DTranspose(filters=32, kernel_size=(3, 3), strides=2, activation='elu', padding='same'))
    decoder.add(L.Conv2DTranspose(filters=3, kernel_size=(3, 3), strides=2, activation=None, padding='same'))
    
    return encoder, decoder


encoder, decoder = build_deep_autoencoder(IMG_SHAPE, code_size=32)
encoder.summary()
decoder.summary()

### Параметры и обучение модели

In [ ]:
inp = L.Input(IMG_SHAPE)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = tf.keras.models.Model(inputs=inp, outputs=reconstruction)
autoencoder.compile(optimizer="adamax", loss='mse')
autoencoder.fit(x=train_data, y=train_data, epochs=10, verbose=1)

In [ ]:
images = train_data
codes = encoder.predict(images) 
assert len(codes) == len(images)

### Построение модели подобия изображений при помощи K ближайших соседей (NearestNeighbours)

In [ ]:
from sklearn.neighbors import NearestNeighbors
nei_clf = NearestNeighbors(metric="euclidean")
nei_clf.fit(codes)

### Применение функций для вывода похожих изображений

In [ ]:
def get_similar(image, n_neighbors=5):
    assert image.ndim==3,"image must be [batch,height,width,3]"
    code = encoder.predict(image[None])    
    (distances,),(idx,) = nei_clf.kneighbors(code,n_neighbors=n_neighbors)
    return distances,images[idx]
def show_similar(image):
    distances,neighbors = get_similar(image,n_neighbors=3)
    plt.figure(figsize=[8,7])
    plt.subplot(1,4,1)
    plt.imshow(image)
    plt.title("Original image")
    
    for i in range(3):
        plt.subplot(1,4,i+2)
        plt.imshow(neighbors[i])
        plt.title("Dist=%.3f"%distances[i])
   	    plt.show()

## Использование предобученных моделей для извлечения признаков из изображения

In [ ]:
# Загрузка весов модели
model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)
model.summary()

In [ ]:
def load_image(path):
    img = image.load_img(path, target_size=model.input_shape[1:3])
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [ ]:
# Использование модели
feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
feat_extractor.summary()

In [ ]:
import time
tic = time.perf_counter()
features = []
for i, image_path in enumerate(filelist[:200]):
    if i % 500 == 0:
        toc = time.perf_counter()
        elap = toc-tic;
        print("analyzing image %d / %d. Time: %4.4f seconds." % (i, len(images),elap))
        tic = time.perf_counter()
    img, x = load_image(path);
    feat = feat_extractor.predict(x)[0]
    features.append(feat)
print('finished extracting features for %d images' % len(images))

from sklearn.decomposition import PCA
features = np.array(features)
pca = PCA(n_components=100)
pca.fit(features)

pca_features = pca.transform(features)

In [ ]:
# Поиск похожих изображений
from scipy.spatial import distance
similar_idx = [ distance.cosine(pca_features[80], feat) for feat in pca_features ]

idx_closest = sorted(range(len(similar_idx)), key=lambda k: similar_idx[k])[1:6] # отображение первых 6 похожих изображений

thumbs = []
for idx in idx_closest:
    img = image.load_img(filelist[idx])
    img = img.resize((int(img.width * 100 / img.height), 100))
    thumbs.append(img)

# concatenate the images into a single image
concat_image = np.concatenate([np.asarray(t) for t in thumbs], axis=1)

# show the image
plt.figure(figsize = (16,12))
plt.imshow(concat_image)

## Использование готовых библиотек

### Функция для перевода изображений в вектор

In [ ]:
# На выходе выдает словарь с векторами изображений и список "плохих изображений", для которых вектор не определился
def get_vector(train_image):
    diff = {}
    bad = []
    for image in tqdm(train_image):
        try:
            img = face_recognition.load_image_file(image)
            img_enc = face_recognition.face_encodings(img)[0]
            diff.update({image:img_enc})
        except IndexError:
            bad.append(image)
    return diff, bad

In [ ]:
# Применение функции
r, bf = get_vector(photo)

### Функция для кодирования изображения в вектор и сравнения каждого изображения со всем набором изображений в датасете

In [ ]:
def compare_faces(test_image, train_images):
    img1 = face_recognition.load_image_file(test_image)
    img1_enc = face_recognition.face_encodings(img1)[0]
    print('Original_image:')
    print(Path(test_image).stem)
    Image.fromarray(img1).show()
    print('Compared images:')
    differences = {}
    for name,vec in tqdm(train_images.items()):
        try:
            result = face_recognition.compare_faces([img1_enc], vec, tolerance=0.49)
            differences.update({name:result})
        except IndexError:
            pass            
    new_df = {key:value for key,value in differences.items() if value == [True]}
    fig = plt.figure(figsize=(15,len(new_df.keys())))
    rows,cols = 1, len(new_df.keys())
    for idx, i in enumerate(new_df.keys()):
        #if Path(i).stem[:20] != Path(i+1).stem[:20]:
        fig.add_subplot(rows, cols, idx+1)
        im = Image.open(i)
        print(Path(i).stem)
        plt.imshow(im)
        plt.axis(False)

In [ ]:
# Применение функции
compare_faces(photo[9], r)

### Сохранение похожих изображений в Excel-файл

### Функция для отображения похожих изображений

In [ ]:
def get_true_images(test_image, train_image):
    names = {}
    for t in tqdm(test_image):
        differences = {}
        try:
            img1 = face_recognition.load_image_file(t)
            img1_enc = face_recognition.face_encodings(img1)[0]
        except IndexError:
            print(t)
        for name, vector in train_image.items():
            try:
                result = face_recognition.compare_faces([img1_enc], vector, tolerance=0.4)
                differences.update({name:result})
            except IndexError:
                pass
        new_df = {key:value for key,value in differences.items() if value == [True]}
        names.update({t:list(new_df.keys())})
    return names

In [ ]:
# Применение функции
dictionary = get_true_images(photo, r)
#dictionary

### Функция для составления датасета из набора похожих изображений

In [ ]:
def get_names(dictionary):
    new_list = {}
    for idx, i in enumerate(list(dictionary.keys())):
        b = Path(i).stem
        stem = []
        for j in list(dictionary.values())[idx]:
            a = Path(j).stem
            stem.append(a)
        new_list.update({b:stem})
    data = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in new_list.items()]))
    return data

In [ ]:
# Применение функции
d = get_names(dictionary)

In [ ]:
# Сохранение файла в EXCEL - при необходимости. На данном этапе данные с дублями
d.to_excel('find_faces.xlsx', sheet_name = 'Test')